In [1]:
from fastscoredeploy import ipmagic

In [2]:
%%schema double
{"type": "double"}

Schema loaded and bound to double variable


In [3]:
%%model tfmodel
# fastscore.schema.0: double
# fastscore.schema.1: double
# fastscore.module-attached: tensorflow

import tensorflow as tf
import numpy as np

def begin():
    tf.logging.set_verbosity(tf.logging.WARN)
    global past_days
    past_days = []

    global n_steps, X, outputs, sess
    n_steps = 30
    n_inputs = 1
    n_outputs = 1
    n_layers = 3
    n_units = 100
    save_name = 'tf_sp500_lstm'

    X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
    y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
    layers = [tf.contrib.rnn.BasicLSTMCell(num_units = n_units) for k in range(n_layers)]
    cell = tf.contrib.rnn.OutputProjectionWrapper(
      tf.contrib.rnn.MultiRNNCell(layers), output_size = n_outputs)
    outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    saver = tf.train.Saver()
    sess = tf.Session()
    saver.restore(sess, './{}'.format(save_name))

def predict(inputs):
    if len(inputs[0,:,0]) < n_steps:
        raise Exception("Insufficient inputs! X should be an array of shape (1, k, 1) where k >= n_steps")

    preds = []
    steps_forward = len(inputs[0,:,0]) - n_steps + 1
    for j in range(0, steps_forward):
        X_in = inputs[:,j:j+n_steps,:]
        y_pred = sess.run(outputs, feed_dict = {X: X_in})
        preds.append(y_pred[0,-1,0]) # add the last one
    return np.array(preds).reshape(-1, len(preds), 1)

def action(x):
    global past_days
    past_days.append(x)
    past_days = past_days[-n_steps:]
    if len(past_days) == n_steps:
        result = predict(np.array(past_days).reshape(-1, n_steps, 1))[0, 0, 0]
        yield np.asscalar(result)
    else:
        yield x

def end():
    sess.close()


/opt/conda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Model loaded and bound to tfmodel variable.


In [4]:
tfmodel.mtype

'python3'

In [5]:
from fastscoredeploy.suite import Connect

connect = Connect("https://dashboard:8000")
engine = connect.lookup('engine')
mm = connect.lookup('model-manage')

In [6]:
connect.fleet()

[{'api': 'composer',
  'built_on': 'Wed Jan 24 16:04:03 UTC 2018',
  'health': 'ok',
  'host': 'composer',
  'id': '4f0348c2-060a-11e8-a765-02420a000011',
  'name': 'composer-1',
  'port': 8010,
  'release': '0.1'}, {'api': 'engine',
  'built_on': 'Mon Jan 29 15:38:47 UTC 2018',
  'health': 'ok',
  'host': 'engine',
  'id': '090d0ce4-50d8-43ec-a003-e5de28ed512b',
  'name': 'engine-1',
  'port': 8003,
  'release': '1.6.1'}, {'api': 'model-manage',
  'built_on': 'Thu Nov  9 16:01:58 UTC 2017',
  'health': 'ok',
  'host': 'model-manage',
  'id': 'edd91243-9f72-4a10-83ad-d6c308f285cb',
  'name': 'model-manage-1',
  'port': 8002,
  'release': '1.6.1'}]

In [7]:
tfmodel.update(model_manage=mm)

False

In [8]:
from fastscore.attachment import Attachment
att = Attachment('att', datafile='attachment.tar.gz')
att.upload(tfmodel)

In [9]:
tfmodel.deploy(engine)

In [10]:
import json

input_data = []

with open('test_data.jsons', 'r') as f:
    for line in f:
        try:
            input_data.append(json.loads(line))
        except:
            pass

In [11]:
engine.score(input_data[:60])

[2.1772143840789795,
 2.1678619384765625,
 2.170048952102661,
 2.1543917655944824,
 2.1880810260772705,
 2.095344305038452,
 2.1357288360595703,
 1.9873629808425903,
 2.02543044090271,
 1.8540129661560059,
 1.74863862991333,
 1.8091214895248413,
 1.7126928567886353,
 1.7642762660980225,
 1.7859008312225342,
 1.6083106994628906,
 1.7725034952163696,
 1.7951457500457764,
 1.899043321609497,
 1.6914918422698975,
 1.7033576965332031,
 1.687721848487854,
 1.5650486946105957,
 1.4746744632720947,
 1.48747980594635,
 1.6409997940063477,
 1.629782795906067,
 1.6451754570007324,
 1.7244477272033691,
 1.712424635887146,
 1.7961549758911133]

In [12]:
engine.reset()